In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.distributions.categorical import Categorical
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam
import numpy as np
import wandb
import os
import random
from time import time

from lib.data import EpisodeDataset
from lib.neural_networks import FullyConnected3, ConvNet

In [2]:
WIDTH = 10
OBS_SIZE = 46
HIDDEN_SIZE = 512

In [47]:
TRAIN_PATH_1 = '/home/bdemoss/Desktop/RL IDM/Episode Data/Random Data/30 TPS/3v3 Train'
TRAIN_PATH_2 = '/home/bdemoss/Desktop/RL IDM/Episode Data/Nexto Data/30 TPS/3v3 Train'

train_dataset = EpisodeDataset([TRAIN_PATH_1], width=WIDTH, include_change=True)

print(len(train_dataset))

Loading Episode Data
Finished Loading Episode Data
182160


In [48]:
Train_Data_Loader = DataLoader(train_dataset, batch_size=1, shuffle=True, num_workers=16)

In [5]:
#model = ConvNet(obs_size=OBS_SIZE, obs_width=WIDTH, conv_number=10, hidden_size=HIDDEN_SIZE)
model = FullyConnected3(obs_size=OBS_SIZE, obs_width=WIDTH, hidden_size=HIDDEN_SIZE)

gpumodel = model.cuda()

criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(gpumodel.parameters(), lr=0.0001)

In [53]:
gpumodel.load_state_dict(torch.load('trained_networks/fc3_pitch_512_w_change.pt'))

<All keys matched successfully>

In [49]:
wandb.init(project="Inverse Dynamics Model", entity="harrymead")

wandb.config = {
  "learning_rate": 0.0001,
  "epochs": 10,
  "batch_size": 10
}

accuracy/boost,▁▅▅▅▇▆▆▇▆▇▇▇▇▇▇▇▇██▇▇▇▇▇█▇██▇█▇█▇▇██████
accuracy/handbrake,▁▃▄▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇█▇▇██▇██▇██▇█▇████
accuracy/jump,▁▂▃▅▅▆▅▅▅▆▆▆▆▆▆▆▆▆▇▇▇▆▇▇▆▆▇▇▇▇▇█▇▇▇▇█▇█▇
accuracy/pitch,▁▄▄▆▆▆▆▆▇▇▇▇▇▇▇▇▇██▇▇██▇█▇███████▇██████
accuracy/roll,▁▃▄▅▅▅▅▅▆▆▆▇▇▆▇▇▇▇█▇▇▇▇▇█▇██▇███▇▇█████▇
accuracy/steer,▁▄▄▅▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇████▇█▇▇██████
accuracy/throttle,▁▅▅▆▇▇▆▇▇▇▇▇█▇▇▇███████▇████████████████
accuracy/yaw,▁▃▄▅▅▆▆▆▆▇▆▇▇▆▇▇▇▇▇▇▇▇▇▆▇▇▇█▇█▇▇▇▇█▇▇▇█▇
loss,█▅▅▄▃▃▃▃▂▂▂▂▂▂▂▂▂▁▁▁▁▂▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
accuracy/boost,0.98607
accuracy/handbrake,0.99295


In [50]:
gpumodel.train()

for epoch in range(10):
    batch = 0
    total_loss = 0
    obs_count = 0
    total_correct = torch.zeros(8).cuda()

    for _obs, _act, _add_data in Train_Data_Loader:
        # TRAINING 
        obs = _obs[0].cuda()
        act = _act[0].long().cuda()
        add_data = _add_data[0]

        if obs.shape[0] == 0:
            continue

        optimizer.zero_grad()

        y_pred = model(obs)
        pred_act = torch.zeros(obs.shape[0], 8).cuda()

        loss = 0
        for i in range(8):
            loss += criterion(y_pred[i], act[:, i])

            pred_act[:, i] = torch.argmax(y_pred[i], dim=1)

        loss.backward()
        optimizer.step()


        # LOGGING
        obs_count += len(obs)
        total_loss += loss.item() * len(obs)

        correct = act == pred_act
        total_correct += torch.sum(correct, dim=0)


        batch += 1
        if batch % 1000 == 0:
            idv_accuracy = total_correct/obs_count

            print(f'Epoch {epoch + 1} - Loss: {total_loss / obs_count}, Correct: {idv_accuracy}')
            wandb.log({"loss": total_loss/obs_count, "accuracy/throttle": idv_accuracy[0], "accuracy/steer": idv_accuracy[1], "accuracy/yaw": idv_accuracy[2], 
                        "accuracy/pitch": idv_accuracy[3], "accuracy/roll": idv_accuracy[4], "accuracy/jump": idv_accuracy[5], "accuracy/boost": idv_accuracy[6], "accuracy/handbrake": idv_accuracy[7]})

            total_loss = 0
            obs_count = 0
            total_correct = torch.zeros(8).cuda()

Epoch 1 - Loss: 0.5110927663154049, Correct: tensor([0.9909, 0.9572, 0.9465, 0.9566, 0.9677, 0.9847, 0.9893, 1.0000],
       device='cuda:0')
Epoch 1 - Loss: 0.5323511604712086, Correct: tensor([0.9905, 0.9545, 0.9446, 0.9535, 0.9644, 0.9843, 0.9893, 0.9999],
       device='cuda:0')
Epoch 1 - Loss: 0.54111928258806, Correct: tensor([0.9914, 0.9525, 0.9435, 0.9527, 0.9645, 0.9828, 0.9889, 1.0000],
       device='cuda:0')
Epoch 1 - Loss: 0.5276777908594452, Correct: tensor([0.9901, 0.9535, 0.9460, 0.9537, 0.9673, 0.9837, 0.9876, 1.0000],
       device='cuda:0')
Epoch 1 - Loss: 0.5144414948485708, Correct: tensor([0.9913, 0.9554, 0.9449, 0.9563, 0.9668, 0.9861, 0.9893, 1.0000],
       device='cuda:0')
Epoch 1 - Loss: 0.5184007354675029, Correct: tensor([0.9909, 0.9577, 0.9446, 0.9558, 0.9664, 0.9844, 0.9884, 1.0000],
       device='cuda:0')
Epoch 1 - Loss: 0.5024329682680508, Correct: tensor([0.9918, 0.9567, 0.9477, 0.9563, 0.9684, 0.9835, 0.9899, 1.0000],
       device='cuda:0')
Epoch 1 

KeyboardInterrupt: 

In [55]:
torch.save(gpumodel.state_dict(), "trained_networks/fc3_all_512_w_change_random.pt")

In [9]:
#TEST_PATH = '/home/bdemoss/Desktop/RL IDM/Episode Data/Nexto Data/30 TPS/3v3 Test'
TEST_PATH = '/home/bdemoss/Desktop/RL IDM/Episode Data/Human Example Data/30 TPS'

test_dataset = EpisodeDataset([TEST_PATH], width=WIDTH, include_change=True)

Loading Episode Data
Finished Loading Episode Data


In [51]:
Test_Data_Loader = DataLoader(test_dataset, batch_size=1, shuffle=False, num_workers=0)

In [56]:
conf_mtx = torch.zeros((8,3,3)).cuda()
CONF_MAT = True

gpumodel.eval()

total_loss = 0
obs_count = 0
total_correct = torch.zeros(8).cuda()

correct_on_ground = torch.zeros(8).cuda()
total_on_ground = 0

for _obs, _act, _add_data in Test_Data_Loader:
    # EVALUATION
    obs = _obs[0].cuda()
    act = _act[0].long().cuda()
    add_data = _add_data[0].long()

    if obs.shape[0] == 0:
        continue

    with torch.no_grad():
        y_pred = model(obs)

    y_pred = model(obs)
    pred_act = torch.zeros(obs.shape[0], 8).cuda()

    loss = 0
    for i in range(8):
        loss += criterion(y_pred[i], act[:, i])

        pred_act[:, i] = torch.argmax(y_pred[i], dim=1)


    # LOGGING
    obs_count += len(obs)
    total_loss += loss.item() * len(obs)

    correct = act == pred_act
    total_correct += torch.sum(correct, dim=0)

    on_ground = add_data[:,0]

    #print(correct)
    #print(on_ground.bool())

    total_on_ground += torch.sum(on_ground == 0).item()


    correct_on_ground += torch.sum(correct[on_ground == 0], dim=0)

    if CONF_MAT:
        for i in range(obs.shape[0]):
            for j in range(8):
                if on_ground[i] == 0:
                    conf_mtx[j, act[i, j], pred_act[i, j].long()] += 1

    

print(f'Loss: {total_loss / obs_count}, Correct: {total_correct / obs_count}, Correct On Ground: {correct_on_ground / total_on_ground}, On Ground: {total_on_ground / obs_count}')

np.set_printoptions(precision=4, suppress=True)
print((conf_mtx).cpu().numpy())

Loss: 6.932987279707397, Correct: tensor([0.9714, 0.9660, 0.9401, 0.9646, 0.9807, 0.9670, 0.9941, 0.9683],
       device='cuda:0'), Correct On Ground: tensor([0.9968, 0.9451, 0.8646, 0.9435, 0.9568, 0.9296, 0.9971, 1.0000],
       device='cuda:0'), On Ground: 0.4396306818181818
[[[    0.     0.     0.]
  [    1. 13881.    52.]
  [    0.     7.  4629.]]

 [[ 1984.    58.    16.]
  [  512. 13540.   314.]
  [   29.    90.  2027.]]

 [[ 4467.   424.   103.]
  [ 1163.  7265.   584.]
  [   65.   175.  4324.]]

 [[ 1964.    73.    21.]
  [  498. 13516.   352.]
  [   29.    76.  2041.]]

 [[ 2670.   132.    35.]
  [  281. 11847.   265.]
  [   27.    62.  3251.]]

 [[15622.   346.     0.]
  [  962.  1640.     0.]
  [    0.     0.     0.]]

 [[13891.    43.     0.]
  [   10.  4626.     0.]
  [    0.     0.     0.]]

 [[18570.     0.     0.]
  [    0.     0.     0.]
  [    0.     0.     0.]]]
